In [1]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install mediapipe

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-o8qmn345
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-o8qmn345
  Resolved https://github.com/huggingface/transformers.git to commit dd16acb8a3e93b643aa374c9fb80749f5235c1a6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.49.0.dev0-py3-none-any.whl size=10770282 sha256=2c19917b82715c3eb90426a015d8b32747e886f6471d3f9dc9ce62d5dc107bc0
  Stored in directory: /tmp/pip-ephem-wheel-cache-n03qkc_q/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import cv2
import numpy as np
import torch
from transformers import DepthProImageProcessorFast, DepthProForDepthEstimation
import mediapipe as mp
from PIL import Image
from google.colab import files
import gc
gc.collect()
torch.cuda.empty_cache()


In [3]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load DepthPro model and processor
image_processor = DepthProImageProcessorFast.from_pretrained("apple/DepthPro-hf")
model = DepthProForDepthEstimation.from_pretrained("apple/DepthPro-hf").to(device)

# Initialize Mediapipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.92k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.90G [00:00<?, ?B/s]

In [4]:
def estimate_depth_and_focal_length(frame):
    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    inputs = image_processor(images=pil_image, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    post_processed_output = image_processor.post_process_depth_estimation(
        outputs, target_sizes=[(pil_image.height, pil_image.width)]
    )

    depth_map = post_processed_output[0]["predicted_depth"].detach().cpu().numpy()
    focal_length = post_processed_output[0].get("focal_length", None)

    if focal_length is not None:
        # Move focal_length to CPU and convert to scalar value
        focal_length = focal_length.detach().cpu().item()  # Convert to scalar value

    return depth_map, focal_length

In [5]:
def get_3d_coordinates_pinhole_camera_model(landmark, depth_map, image_shape):
    height, width = image_shape[:2]
    x, y = int(landmark.x * width), int(landmark.y * height)
    if 0 <= y < height and 0 <= x < width:
        depth = depth_map[y, x]
    else:
        return None
    fx, fy, cx, cy = width / 2, height / 2, width / 2, height / 2
    z = depth
    x_3d = (x - cx) * z / fx
    y_3d = (y - cy) * z / fy
    return np.array([x_3d, y_3d, z])

In [6]:
def get_3d_coordinates_avoiding_calibration(landmark, depth_map, image_shape):
    height, width = image_shape[:2]
    x, y = int(landmark.x * width), int(landmark.y * height)

    if 0 <= y < height and 0 <= x < width:
        depth = depth_map[y, x]
    else:
        return None

    # Normalize the 2D coordinates and scale by depth
    x_3d = (landmark.x - 0.5) * 2 * depth  # Scale x by depth
    y_3d = (landmark.y - 0.5) * 2 * depth  # Scale y by depth
    z_3d = depth  # Depth is the z-coordinate

    return np.array([x_3d, y_3d, z_3d])

In [7]:
def get_3d_coordinates_using_estimated_focal_length(landmark, depth_map, image_shape, focal_length):
    height, width = image_shape[:2]
    x, y = int(landmark.x * width), int(landmark.y * height)

    if 0 <= y < height and 0 <= x < width:
        depth = depth_map[y, x]
    else:
        return None

    fx = fy = focal_length  # Use the estimated focal length
    cx, cy = width / 2, height / 2  # Principal point (center of the image)

    z = depth
    x_3d = (x - cx) * z / fx
    y_3d = (y - cy) * z / fy

    return np.array([x_3d, y_3d, z])

In [8]:
def estimate_focal_length(shoulder_width_pixels, shoulder_depth, shoulder_width_cm=60):
    """
    Estimate the focal length using the shoulder width and depth.
    :param shoulder_width_pixels: Width of the shoulders in pixels.
    :param shoulder_depth: Depth (distance from the camera) of the shoulders in cm.
    :param shoulder_width_cm: Real-world shoulder width in cm (default: 60 cm).
    :return: Estimated focal length in pixels.
    """
    if shoulder_depth <= 0:
        print("Warning: Invalid shoulder depth detected.")
        return None
    focal_length = (shoulder_width_pixels * shoulder_depth) / shoulder_width_cm
    return focal_length

In [9]:
def estimate_focal_length_from_shoulders(results, depth_map, image_shape, shoulder_width_cm=60):
    """
    Estimate the focal length using the shoulders' width and depth.

    :param results: Mediapipe Pose results containing landmarks.
    :param depth_map: Depth map for the current frame.
    :param image_shape: Shape of the image (height, width).
    :param shoulder_width_cm: Real-world shoulder width in cm (default: 60 cm).
    :return: Estimated focal length in pixels or None if landmarks are invalid.
    """
    height, width = image_shape[:2]

    # Get the 2D landmarks of the shoulders
    left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
    right_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]

    # Convert shoulder landmarks to pixel coordinates
    left_shoulder_2d = (int(left_shoulder.x * width), int(left_shoulder.y * height))
    right_shoulder_2d = (int(right_shoulder.x * width), int(right_shoulder.y * height))

    # Calculate shoulder width in pixels
    shoulder_width_pixels = np.linalg.norm(np.array(left_shoulder_2d) - np.array(right_shoulder_2d))

    # Get shoulder depth (average depth of left and right shoulders)
    left_shoulder_depth = depth_map[int(left_shoulder.y * height), int(left_shoulder.x * width)]
    right_shoulder_depth = depth_map[int(right_shoulder.y * height), int(right_shoulder.x * width)]

    # Ensure valid depth values
    if left_shoulder_depth <= 0 or right_shoulder_depth <= 0:
        print("Warning: Invalid shoulder depth detected.")
        return None

    shoulder_depth = (left_shoulder_depth + right_shoulder_depth) / 2

    # Estimate focal length using shoulder width and depth
    focal_length = estimate_focal_length(shoulder_width_pixels, shoulder_depth, shoulder_width_cm)

    return focal_length

In [10]:
# Method 1: Calculate foot distance directly using depth map based on pinhole camera model or avoiding calibration
def calculate_3D_euclidean_distance(left_foot, right_foot, depth_map, image_shape, get_3d_coordinates_method):
    left_foot_3d = get_3d_coordinates_method(left_foot, depth_map, image_shape)
    right_foot_3d = get_3d_coordinates_method(right_foot, depth_map, image_shape)
    if left_foot_3d is not None and right_foot_3d is not None:
        foot_distance_3d = np.linalg.norm(left_foot_3d - right_foot_3d)
        return foot_distance_3d * 100  # Convert to centimeters
    return None

In [11]:
# Method 2: Calculate foot distance using focal length and depth map based on estimated focal length by the model
def calculate_foot_distance_method_using_estimated_focal_length_and_depth_map(left_foot, right_foot, depth_map, image_shape, focal_length):
    left_foot_3d = get_3d_coordinates_using_estimated_focal_length(left_foot, depth_map, image_shape, focal_length)
    right_foot_3d = get_3d_coordinates_using_estimated_focal_length(right_foot, depth_map, image_shape, focal_length)

    if left_foot_3d is not None and right_foot_3d is not None:
        foot_distance_3d = np.linalg.norm(left_foot_3d - right_foot_3d)
        return foot_distance_3d * 100  # Convert to centimeters
    return None


In [12]:
# Process each frame and test both methods
def process_video(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        frame_count = 0
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("End of video or cannot read frame.")
                break

            # Estimate depth for the current frame
            depth_map, focal_length = estimate_depth_and_focal_length(image)

            # Process the image with Mediapipe Pose
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.pose_landmarks:
                # Draw pose landmarks
                mp_drawing.draw_landmarks(
                    image,
                    results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),
                )

                # Get the 2D landmarks of the feet
                left_foot = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_FOOT_INDEX]
                right_foot = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX]

                # Convert 2D landmarks to pixel coordinates
                height, width, _ = image.shape
                left_foot_2d = (int(left_foot.x * width), int(left_foot.y * height))
                right_foot_2d = (int(right_foot.x * width), int(right_foot.y * height))

                # Draw a line between the two feet
                cv2.line(image, left_foot_2d, right_foot_2d, (0, 255, 0), 2)  # Green line

                estimated_focal_length_from_shoulders = estimate_focal_length_from_shoulders(results, depth_map, image.shape, shoulder_width_cm=60)
                # Calculate foot distances using both methods
                method_1_distance = calculate_3D_euclidean_distance(left_foot, right_foot, depth_map, image.shape , get_3d_coordinates_method = get_3d_coordinates_pinhole_camera_model)
                method_2_distance = calculate_3D_euclidean_distance(left_foot, right_foot, depth_map, image.shape , get_3d_coordinates_method = get_3d_coordinates_avoiding_calibration)
                method_3_distance = calculate_foot_distance_method_using_estimated_focal_length_and_depth_map(left_foot, right_foot, depth_map, image.shape , focal_length)
                method_4_distance = calculate_foot_distance_method_using_estimated_focal_length_and_depth_map(left_foot, right_foot, depth_map, image.shape , estimated_focal_length_from_shoulders)
                # Display the distances on the image
                y_position = 50  # Starting vertical position for text
                line_spacing = 50  # Spacing between lines of text

                if method_1_distance is not None:
                    cv2.putText(
                        image,
                        f"Method 1 (Pinhole Camera Model): {method_1_distance:.2f} cm",
                        (50, y_position),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (0, 255, 0),  # Green color for Method 1
                        2,
                    )
                    y_position += line_spacing  # Move to the next line

                if method_2_distance is not None:
                    cv2.putText(
                        image,
                        f"Method 2 (Avoiding Calibration): {method_2_distance:.2f} cm",
                        (50, y_position),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (255, 255, 0),  # Cyan color for Method 2
                        2,
                    )
                    y_position += line_spacing  # Move to the next line

                if method_3_distance is not None:
                    cv2.putText(
                        image,
                        f"Method 3 (Estimated Focal Length + Depth Map): {method_3_distance:.2f} cm",
                        (50, y_position),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (255, 0, 0),  # Blue color for Method 3
                        2,
                    )
                    y_position += line_spacing  # Move to the next line

                if method_4_distance is not None:
                    cv2.putText(
                        image,
                        f"Method 4 (Focal Length from Shoulder Proportions): {method_4_distance:.2f} cm",
                        (50, y_position),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (0, 0, 255),  # Red color for Method 4
                        2,
                    )

                # Write the processed frame to the output video
                out.write(image)

                # Print progress
                frame_count += 1
                if frame_count % 10 == 0:
                    print(f"Processed {frame_count} frames.")

    # Release resources
    cap.release()
    out.release()
    print("Processing complete.")

In [13]:
# Example usage
video_path = '/content/Kirolos_video.mp4'  # Replace with your video path
output_path = '/content/output_with_distances.mp4'
process_video(video_path, output_path)

Processed 10 frames.
Processed 20 frames.
Processed 30 frames.
Processed 40 frames.
Processed 50 frames.
Processed 60 frames.
Processed 70 frames.
Processed 80 frames.
Processed 90 frames.
Processed 100 frames.
Processed 110 frames.
Processed 120 frames.
Processed 130 frames.
Processed 140 frames.
Processed 150 frames.
Processed 160 frames.
Processed 170 frames.
Processed 180 frames.
Processed 190 frames.
Processed 200 frames.
Processed 210 frames.
Processed 220 frames.
Processed 230 frames.
Processed 240 frames.
Processed 250 frames.
Processed 260 frames.
Processed 270 frames.
Processed 280 frames.
Processed 290 frames.
Processed 300 frames.
Processed 310 frames.
Processed 320 frames.
Processed 330 frames.
Processed 340 frames.
Processed 350 frames.
Processed 360 frames.
Processed 370 frames.
Processed 380 frames.
Processed 390 frames.
Processed 400 frames.
Processed 410 frames.
Processed 420 frames.
Processed 430 frames.
Processed 440 frames.
Processed 450 frames.
Processed 460 frame